In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('resources/filtered_train.csv')

In [7]:
df.head()

,restaurant_id,user_id,score
0,2,14,1
1,2,2087,1
2,2,4625,1
3,2,4626,1
4,2,4826,1


In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030232 entries, 0 to 1030231
Data columns (total 3 columns):
restaurant_id    1030232 non-null int64
user_id          1030232 non-null int64
score            1030232 non-null int64
dtypes: int64(3)
memory usage: 23.6 MB


In [4]:
df = df.apply(pd.to_numeric,downcast='unsigned')

In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030232 entries, 0 to 1030231
Data columns (total 3 columns):
restaurant_id    1030232 non-null uint32
user_id          1030232 non-null uint32
score            1030232 non-null uint8
dtypes: uint32(2), uint8(1)
memory usage: 8.8 MB


In [6]:
n_users = df['user_id'].nunique()
n_restaurants = df['restaurant_id'].nunique()
print(n_users)
print(n_restaurants)

168893
136763


In [35]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

user_id_c = CategoricalDtype(sorted(df.user_id.unique()), ordered=True)
restaurant_id_c = CategoricalDtype(sorted(df.restaurant_id.unique()), ordered=True)
df['score'] = df.score.astype('f')
value = df['score']
row = df.user_id.astype(user_id_c).cat.codes
col = df.restaurant_id.astype(restaurant_id_c).cat.codes

In [36]:
sparse_matrix = csr_matrix((value, (row, col)), shape=(user_id_c.categories.size, restaurant_id_c.categories.size))

In [37]:
sparse_matrix

<168893x136763 sparse matrix of type '<class 'numpy.float32'>'
	with 1030232 stored elements in Compressed Sparse Row format>

In [42]:
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(sparse_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

MemoryError: 